# Télécharger en block les données de WaPOR

Dans ce notebook, nous utiliserons le module Python local pour automatiser les étapes de lecture des métadonnées de l'ensemble de données, demander l'URL de téléchargement, télécharger l'ensemble de données raster, prétraiter l'ensemble de données raster en utilisant les informations des métadonnées (telles que le facteur de conversion, l'unité). **Les données téléchargées par ce script seront automatiquement corrigées par le facteur de conversion**.

Exécutez la cellule de code ci-dessous pour importer les packages et les modules nécessaires. Il vous sera demandé de fournir la clé API WaPOR (voir [Start here](0_Start_here.ipynb) la page Démarrer ici pour savoir comment obtenir la clé API). Le module **WaPOR** module gardera votre clé en mémoire et l'enregistrera dans le dossier du module. Si vous souhaitez utiliser une nouvelle clé API, vous devez supprimer ce fichier 
**'modules\WaPOR\wapor_api_token.pickle'**

In [1]:
import os
import glob
import sys
import shapefile             
import matplotlib.pyplot as plt 

folder=r"..\..\modules"
sys.path.append(folder) #ajouter un dossier avec des modules locaux aux chemins du système
import WaPOR #importer les modules locaux de 'WaPOR'
WaPOR.API.getCatalog()

Loading WaPOR catalog...
Loading WaPOR catalog...Done


,code,caption,description,additionalInfo,tags,workspaceCode,dataType,index,operation,hidden,links,measure,dimension
0,L1_GBWP_A,Gross Biomass Water Productivity,The annual Gross Biomass Water Productivity ex...,"{'format': 'Raster Dataset', 'unit': 'kg/m³ is...","[{'name': 'L1', 'index': 1, 'context': 'LEVELS...",WAPOR_2,RASTER,101,"{'pixelTimeSeries': True, 'areaStatsTimeSeries...",False,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'WPR', 'caption': 'Water Productivity...","[{'code': 'YEAR', 'caption': 'Year', 'descript..."
1,L1_NBWP_A,Net Biomass Water Productivity,The annual Net Biomass Water Productivity expr...,"{'format': 'Raster Dataset', 'unit': 'kg/m³ is...","[{'name': 'L1', 'index': 2, 'context': 'LEVELS...",WAPOR_2,RASTER,102,"{'pixelTimeSeries': True, 'areaStatsTimeSeries...",False,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'WPR', 'caption': 'Water Productivity...","[{'code': 'YEAR', 'caption': 'Year', 'descript..."
2,L1_AETI_A,Actual EvapoTranspiration and Interception (An...,The actual EvapoTranspiration and Interception...,"{'format': 'Raster Dataset', 'unit': 'mm', 'da...","[{'name': 'L1', 'index': 3, 'context': 'LEVELS...",WAPOR_2,RASTER,103,"{'pixelTimeSeries': True, 'areaStatsTimeSeries...",False,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'WATER_MM', 'caption': 'Amount of Wat...","[{'code': 'YEAR', 'caption': 'Year', 'descript..."
3,L1_AETI_M,Actual EvapoTranspiration and Interception (Mo...,The actual EvapoTranspiration and Interception...,"{'format': 'Raster Dataset', 'unit': 'mm', 'da...","[{'name': 'L1', 'index': 4, 'context': 'LEVELS...",WAPOR_2,RASTER,104,"{'pixelTimeSeries': True, 'areaStatsTimeSeries...",False,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'WATER_MM', 'caption': 'Amount of Wat...","[{'code': 'MONTH', 'caption': 'Month', 'worksp..."
4,L1_AETI_D,Actual EvapoTranspiration and Interception (De...,The actual EvapoTranspiration and Interception...,"{'format': 'Raster Dataset', 'unit': 'mm', 'da...","[{'name': 'L1', 'index': 5, 'context': 'LEVELS...",WAPOR_2,RASTER,105,"{'pixelTimeSeries': True, 'areaStatsTimeSeries...",False,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'WATER_MM', 'caption': 'Amount of Wat...","[{'code': 'DEKAD', 'caption': 'Dekad (10-Days ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,L2_BAS_PHE_S,"Phenology (Seasonal, clipped by basin)","This is a system dataset, used to let the down...",NaN,NaN,WAPOR_2,RASTER,4151,"{'pixelTimeSeries': False, 'areaStatsTimeSerie...",True,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'PHE', 'caption': 'Phenology', 'descr...","[{'code': 'YEAR', 'caption': 'Year', 'descript..."
194,L2_BAS_QUAL_NDVI_D,Quality of Normalized Difference Vegetation In...,"This is a system dataset, used to let the down...",NaN,NaN,WAPOR_2,RASTER,4161,"{'pixelTimeSeries': False, 'areaStatsTimeSerie...",True,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'N_DAYS', 'caption': 'Number of Days'...","[{'code': 'DEKAD', 'caption': 'Dekad (10-Days ..."
195,L2_CTY_QUAL_NDVI_D,Quality of Normalized Difference Vegetation In...,"This is a system dataset, used to let the down...",NaN,NaN,WAPOR_2,RASTER,4162,"{'pixelTimeSeries': False, 'areaStatsTimeSerie...",True,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'N_DAYS', 'caption': 'Number of Days'...","[{'code': 'DEKAD', 'caption': 'Dekad (10-Days ..."
196,L2_BAS_QUAL_LST_D,"Quality Land Surface Temperature (Dekadal, cli...","This is a system dataset, used to let the down...",NaN,NaN,WAPOR_2,RASTER,4171,"{'pixelTimeSeries': False, 'areaStatsTimeSerie...",True,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'N_DAYS', 'caption': 'Number of Days'...","[{'code': 'DEKAD', 'caption': 'Dekad (10-Days ..."


## Dataset code

Le module **WaPOR** contient les functions *download_dekadal*, *download_monthly*, and *download_yearly* qui téléchargent par lots les données WaPOR pour une résolution temporelle décadaire, mensuelle et annuelle. Pour séparer la couche de données à télécharger, ces fonctions nécessitent le code des données, la coordonnée de la boîte de délimentation, le niveau de données et la date de début et de fin de la période. Le code pour chaque ensemble de données est disponible dans le tableau ci-dessous:

| code        | Name          | 
| :---------: |:-------------:| 
| LCC | Classification de couverture du sol     | 
| PCP      | Précipitation      | 
| RET      | Evapotranspiration de référence   | 
| NPP      | Production primaire nette |
| AETI      | Evapotranspiration réelle et Interception |   
| E | Evaporation      |  
| I | Interception      | 
| T | Transpiration      | 
| PHE | Phénologie      | 
| GBWP | Productivité brute de l’eau de la biomasse     | 

Utilisez le code de données sélectionné au lieu de **RET** pour la valeur du paramètre **data** dans les cellules de code ci-dessous.

## Exemple de fonctions du module WaPOR

Les exemples de cellules de code ci-dessous montrent comment le module WaPOR peut être utilisé pour télécharger par lots les données WaPOR: évapotranspiration de référence décadaire, mensuelle et annuelle d'une étendue. L'étendue est définie par la délimitation du fichier de formes du bassin Awash. Les raster seront collectés et prétraités à l'unité correcte mm / période (mm / décade, mm / mois ou mm / an respectivement), et sauvegardées dans le [output folder](data)


### Définir l'étendue du téléchargement et le dossier de sortie

Changer les valeurs xmin, ymin, xmax, ymax à l'étendue du téléchargement.
Si nécessaire changer le chemin d´accès *output_dir* pour l´adresse du dossier où vous voulez sauvegarder les données.

In [2]:
#définir l'étendue à télécharger à partir d'un fichier de formes:
shape_fh=r".\data\Awash_shapefile.shp"
shape=shapefile.Reader(shape_fh)
xmin,ymin,xmax,ymax=shape.bbox

# définir l'étendue à télécharger par coordonnées:
# xmin,ymin,xmax,ymax=(32.65692516077674,-25.16412729789142,32.90420244933813,-25.01450956754679)

output_dir=r'.\data' # dossier pour enregistrer les données

### Télécharger les données journalières

Notez que les données journalières ne sont disponibles que pour l'évapotranspiration et les précipitations de référence de niveau 1

In [3]:
WaPOR.download_daily(output_dir, 
                       data='RET',
                       Startdate='2009-01-01', 
                       Enddate='2009-01-31',
                       latlim=[ymin-0.5, ymax+0.5], 
                       lonlim=[xmin-0.5, xmax+0.5],
                       level=1,                    
                       )


Download WaPOR Level 1 dekadal RET data for the period 2009-01-01 till 2009-01-31
Progress: |██████████████████████████████████████████████████| 100.0% Complete


'.\\data\\WAPOR.v2_daily_L1_RET_E'

### Télécharger les données décadaires

**Notez que:** l'unité de l'ensemble de données de la décade sera convertie en quantité/décade au lieu de la quantité journalière moyenne/jour

In [4]:
WaPOR.download_dekadal(output_dir, 
                       data='RET',
                       Startdate='2009-01-01', 
                       Enddate='2009-12-31',
                       latlim=[ymin-0.5, ymax+0.5], 
                       lonlim=[xmin-0.5, xmax+0.5],
                       level=1, 
                       )


Download WaPOR Level 1 dekadal RET data for the period 2009-01-01 till 2009-12-31
Progress: |██████████████████████████████████████████████████| 100.0% Complete


'.\\data\\WAPOR.v2_dekadal_L1_RET_D'

### Télécharger les données mensuelles

In [5]:
WaPOR.download_monthly(output_dir, 
                       data='RET',
                       Startdate='2009-01-01', 
                       Enddate='2009-12-31', 
                       latlim=[ymin-0.5, ymax+0.5], 
                       lonlim=[xmin-0.5, xmax+0.5],
                       level=1, 
                       )


Download WaPOR Level 1 monthly RET data for the period 2009-01-01 till 2009-12-31
Progress: |██████████████████████████████████████████████████| 100.0% Complete


'.\\data\\WAPOR.v2_monthly_L1_RET_M'

### Télécharger les données annuelles

In [6]:
WaPOR.download_yearly(output_dir, 
                       data='RET',
                       Startdate='2009-01-01', 
                       Enddate='2009-12-31',
                       latlim=[ymin-0.5, ymax+0.5], 
                       lonlim=[xmin-0.5, xmax+0.5],
                       level=1, 
                       )


Download WaPOR Level 1 yearly RET data for the period 2009-01-01 till 2009-12-31
Progress: |██████████████████████████████████████████████████| 100.0% Complete


'.\\data\\WAPOR.v2_yearly_L1_RET_A'

### Télécharger les données saisonnières

**Notez que**: les données saisonnières ne sont disponibles que pour certaines couches (par exemple, la productivité brute de l'eau de la biomasse, la phénologie)

In [7]:
WaPOR.download_seasonal(output_dir,
                       data='GBWP',
                       Startdate='2009-01-01',
                       Enddate='2010-12-31',
                       latlim=[ymin-0.5, ymax+0.5],
                       lonlim=[xmin-0.5, xmax+0.5],
                       level=2,
                       )
WaPOR.download_seasonal(output_dir,
                       data='PHE',
                       Startdate='2009-01-01',
                       Enddate='2010-12-31',
                       latlim=[ymin-0.5, ymax+0.5],
                       lonlim=[xmin-0.5, xmax+0.5],
                       level=2,
                       )


Download WaPOR Level 2 seasonal GBWP data for the period 2009-01-01 till 2010-12-31
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Download WaPOR Level 2 seasonal PHE data for the period 2009-01-01 till 2010-12-31
Progress: |██████████████████████████████████████████████████| 100.0% Complete


'.\\data\\WAPOR.v2_seasonal_L2_PHE_S'

# Exercice

Télécharger les données mensuelles de niveau 1 des précipitations, de l'évapotranspiration et de l'interception réelles et de la classification annuelle de la couverture terrestre de 2009 à 2010 pour la zone délimitée par le fichier de formes ".\data\Awash_shapefile.shp"

Utilisez les fonctions du module **WaPOR** en suivant les exemples donnés ci-dessus.

In [8]:
'''
Ecrivez votre code ici
'''

'\nEcrivez votre code ici\n'